# ELASTIC TOY NETWORK STA

This file first shows how to run an elastic Static Traffic Assignment in dyntapy. It also shows examples of how to run an elastic assignment with tolls.

1. At first, you import all the necessary packages and files from mostly dyntapy. 

In [ ]:
import warnings
warnings.filterwarnings('ignore') # hide warnings
import os
import numpy as np
import sys
sys.path.append("../../..")

from pickle import dump
from dyntapy.supply_data import get_toy_network, relabel_graph
from dyntapy.demand_data import add_centroids, od_graph_from_matrix
from dyntapy.visualization import show_network
from dyntapy.assignments import StaticAssignment

2. Some toy networks are already made available in the original toolbox, so in the example,two-route is used. The first steps are always the same: 

- Create or retrieve a network
- Create and add centroids to network → adds connectors automatically

  These steps can also be visualised in the notebooks by fully running the next block of code. There are now links visible (roads and connectors), green centroids and for both elements when hovering with the cursor over the segments there are also some characteristics displayed in a box like the link ID, the capacity, the free-flow speed or demand between each location in the demand plot.

In [ ]:
# Get network
network = get_toy_network('two-route')
centroid_x = np.array([-1, 3])
centroid_y = np.array([2, 2])
network = add_centroids(network, centroid_x, centroid_y, euclidean=True)
network = relabel_graph(network) 
show_network(network, euclidean=True, notebook= True)


In [ ]:
# Initial OD to start iterating
old_od = np.zeros(4).reshape((2, 2))
old_od[0, 1] = 400
old_graph = od_graph_from_matrix(old_od, centroid_x, centroid_y)

# One STA assignment, to determine A based on B (literature), D (assignment) and P assignment). 
assignment = StaticAssignment(network, old_graph)
result = assignment.run('dial_b')   

In [ ]:
# A matrix
A = 0.05971545
# B matrix
B = 10

In [ ]:
# First iteration 
skims = result.skim
new_od = np.zeros(4).reshape((2, 2))
new_od[0,1] = (A-skims)/B
print(new_od[0,1])
print(abs((new_od - old_od)))

i = 1 
max_iterations = 10
max_deviation_per_od = 5
while np.max(abs((new_od - old_od))) > max_deviation_per_od and i < max_iterations:
    # Perhaps MSA stap
    old_od = new_od
    old_graph =  od_graph_from_matrix(old_od, centroid_x, centroid_y)
    assignment = StaticAssignment(network,old_graph)
    result = assignment.run('dial_b')
    skims = result.skim
    new_od = np.zeros(4).reshape((2, 2))
    new_od[0,1] = (A-skims)/B
    i += 1
    print(abs((new_od - old_od)))
    

-----------------------------------------------------------------------------------------------------------------------
2. HEEDS preparation
As HEEDS uses an STA in every iteration, we like to preprocess things that stay static throughout the process. For the inelastic case this is both the network with centroids and the od_graph, but for the elastic case it is only the network with the centroids.

In [ ]:
# TODO: save original OD matrix as starting point. Otherwise can't modify demand in STA loop. 

HERE = os.path.dirname(os.path.realpath("__file__"))
HEEDS_data_path = HERE+ os.path.sep+os.pardir+os.path.sep+os.pardir+os.path.sep+'data_map' + os.path.sep + "HEEDS_input"
network_path = HEEDS_data_path + os.path.sep + 'network_with_centroids' + os.path.sep + "elastic_toy"
with open(network_path, 'wb') as network_file:
    dump(network, network_file)
    print(f'network saved at f{network_path}')

